In [456]:
from enum import Enum

# Part 1

In [457]:
class Direction(Enum):
    UP = 'UP'
    RIGHT = 'RIGHT'
    DOWN = 'DOWN'
    LEFT = 'LEFT'

In [458]:
def parse_input():
    with open('input.txt', 'r') as file:
        return [list(line.strip()) for line in file]  

In [459]:
def get_obstacles_row_col(grid: list) -> set:
    obstacles_label = '#'
    row_length = len(grid)
    col_length = len(grid[0])
    
    return set((i, j) for i in range(row_length) for j in range(col_length) if grid[i][j] == obstacles_label)

In [460]:
def get_start_row_col(grid: list) -> tuple:
    start_label = '^'
    row_length = len(grid)
    col_length = len(grid[0])
    
    for i in range(row_length):
        for j in range(col_length):
            if grid[i][j] == start_label:
                return (i, j)

In [461]:
def get_next_row_col(current_row_col: tuple, current_direction: Direction) -> tuple:
    current_row = current_row_col[0]
    current_col = current_row_col[1]
    
    if current_direction == Direction.UP:
        return (current_row-1, current_col)
    if current_direction == Direction.RIGHT:
        return (current_row, current_col+1)
    if current_direction == Direction.DOWN:
        return (current_row+1, current_col)
    if current_direction == Direction.LEFT:
        return (current_row, current_col-1)

In [462]:
def should_turn_right(current_row_col: tuple, current_direction: Direction, obstacles_row_col: set) -> bool:
    next_row_col = get_next_row_col(current_row_col, current_direction)
    return next_row_col in obstacles_row_col

In [463]:
def get_next_direction_after_turn_right(current_direction: Direction) -> Direction:
    if current_direction == Direction.UP:
        return Direction.RIGHT
    if current_direction == Direction.RIGHT:
        return Direction.DOWN
    if current_direction == Direction.DOWN:
        return Direction.LEFT
    if current_direction == Direction.LEFT:
        return Direction.UP

In [464]:
def check_is_exited(current_direction: Direction, current_row_col: tuple, row_length: int, col_length: int) -> bool:
    current_row = current_row_col[0]
    current_col = current_row_col[1]
    
    if current_direction == Direction.UP:
        return current_row < 0
    if current_direction == Direction.RIGHT:
        return current_col == col_length
    if current_direction == Direction.DOWN:
        return current_row_col[0] == row_length
    if current_direction == Direction.LEFT:
        return current_col < 0

In [ ]:
def get_guard_visited_row_col(grid: list, start_row_col: tuple, start_direction: Direction) -> set:
    row_length = len(grid)
    col_length = len(grid[0])
    
    current_row_col = start_row_col
    current_direction = start_direction
    obstacles_row_col = get_obstacles_row_col(grid)
    is_exited = False
    
    guard_visited_row_col = set()
    while not is_exited:
        guard_visited_row_col.add(current_row_col)
        
        # turn right and stand still if there is an obstacle in front
        if should_turn_right(current_row_col, current_direction, obstacles_row_col):
            current_direction = get_next_direction_after_turn_right(current_direction)
            continue
        
        # move forward if there is no obstacle in front
        current_row_col = get_next_row_col(current_row_col, current_direction)
        
        # check if the guard has exited the grid
        is_exited = check_is_exited(current_direction, current_row_col, row_length, col_length)
    
    return guard_visited_row_col

In [466]:
def solve_part_1():
    grid = parse_input()
    
    start_row_col = get_start_row_col(grid)
    start_direction = Direction.UP
    guard_visited_row_col = get_guard_visited_row_col(grid, start_row_col, start_direction)
    
    print(f'the guard has visited {len(guard_visited_row_col)} distinct positions')

In [467]:
solve_part_1()

the guard has visited 5242 distinct positions


# Part 2

In [468]:
def check_is_loop(grid: list, new_obstacles_row_col: set, is_start_row_col: bool, start_row_col: tuple, start_direction: Direction) -> bool:
    # the new obstruction can't be placed at the guard's starting position
    if is_start_row_col: 
        return False
    
    row_length = len(grid)
    col_length = len(grid[0])
    
    current_direction = start_direction
    current_row_col = start_row_col
    is_exited = False
    
    guard_visited_row_col_direction = set()
    while not is_exited:    
        current_row_col_direction = (current_row_col, current_direction)
        # is revisit with same direction -> loop occurred
        if current_row_col_direction in guard_visited_row_col_direction:
            return True
        
        guard_visited_row_col_direction.add(current_row_col_direction)
        
        # turn right and stand still if there is an obstacle in front
        if should_turn_right(current_row_col, current_direction, new_obstacles_row_col):
            current_direction = get_next_direction_after_turn_right(current_direction)
            continue
        
        # move forward if there is no obstacle in front
        current_row_col = get_next_row_col(current_row_col, current_direction)
        
        # check if the guard has exited the grid
        is_exited = check_is_exited(current_direction, current_row_col, row_length, col_length)
    
    # is exited -> loop never occurred
    return False 
    

In [469]:
def solve_part_2():
    grid = parse_input()
    
    start_row_col = get_start_row_col(grid)
    start_direction = Direction.UP
    guard_visited_row_col = get_guard_visited_row_col(grid, start_row_col, start_direction)
    
    result = 0
    for row_col in guard_visited_row_col:
        new_obstacles_row_col = get_obstacles_row_col(grid)
        new_obstacles_row_col.add(row_col)
        if check_is_loop(grid, new_obstacles_row_col, row_col == start_row_col, start_row_col, Direction.UP):
            result += 1
    
    print(f'there are {result} different ways to place an obstacle to create a loop')

In [470]:
solve_part_2()

there are 1424 different ways to place an obstacle to create a loop
